# Building a spam filter with Naive Bayes algorithm.

In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by *Tiago A. Almeida and José María Gómez Hidalgo*, and it can be downloaded from the The [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/228/sms+spam+collection).

This project is based on and extends a similar project shared by [DataQuest](https://github.com/dataquestio/solutions/blob/master/Mission433Solutions.ipynb).

We will use **pandas** to manipluate the data and **re** for working with text.

In [2]:
import pandas as pd
import re

## Exploring the Dataset

First we read in the dataset.

In [4]:
sms_data = pd.read_csv('data/SMSSpamCollection', sep='\t', header=None, names = ['Label', 'SMS'])
print(sms_data.shape)
print( sms_data.head() )
print( sms_data.tail() )

(5572, 2)
  Label                                                SMS
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
     Label                                                SMS
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham               Will ü b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name
